In [2]:
#%pip install torch --index-url https://download.pytorch.org/whl/cpu
%pip install datasets python-dotenv
from utils import *


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [43]:

import random
from datasets import load_dataset, Dataset
import json
from openai.types.chat.chat_completion import ChatCompletion
import os


default_or_options = {
    "max_tokens": 8191,

    "extra_body": {
        "provider": {
            "ignore": [
                "Hyperbolic"
            ],
            "require_parameters": True,
        },
    }
}

def switch_roles(dialog: list):
    dialog = json.loads(json.dumps(dialog))
    for i in range(len(dialog)):
        if dialog[i]["role"] == "assistant":
            dialog[i]["role"] = "user"
        elif dialog[i]["role"] == "user":
            dialog[i]["role"] = "assistant"
    return dialog

def generate_domain_action_dialog(entry):
    results = []

    available_models = [
        #'anthropic/claude-3.5-sonnet-20240620',
        #'openai/chatgpt-4o-latest',
        #'openai/gpt-4o-mini-2024-07-18',
        'meta-llama/llama-3.1-405b-instruct',
        'meta-llama/llama-3.1-70b-instruct',
        'meta-llama/llama-3.1-70b-instruct',
        #"mistralai/mistral-nemo",
        "mistralai/mixtral-8x22b-instruct",
        #"mistralai/mixtral-8x22b-instruct",
        #"mistralai/mixtral-8x22b-instruct",
        #"mistralai/mixtral-8x22b-instruct",
        #"mistralai/mistral-small",
        "mistralai/mistral-large"
    ]

    name_lower = entry["name"].lower()
    name_capitalized = ' '.join([word.lower().capitalize() for word in entry["name"].split(' ')])
    action_names = [
        name_lower.replace(" ", "_"),
        name_lower.replace(" ", ""),
        name_lower.replace(" ", "-"),
        name_capitalized.replace(" ", "_"),
        name_capitalized.replace(" ", ""),
        name_capitalized.replace(" ", "-"),
    ]

    model = random.choice(available_models)
    function_name = random.choice(action_names)
    function = {
        "type": "function",
        "function": {
            "name": function_name,
            "parameters": json.loads(entry['parameter_schema']),
        },
    }

    dialog = []

    openrouter_response: ChatCompletion = openrouter.chat.completions.create(
        model=model,
        messages=[{
            "role": "system",
            "content": f"You roleplay an expert {entry['profession']} currently in the process of {entry['process']}. Be creative during this roleplay, you are in the lead, so you need to fill the story with detail.\n"+
                f"Your task is to {entry['name']}. {entry['description']}"
        }, {
            "role": "user",
            "content": f"Please use the following schema to {entry['name']}: {entry['parameter_schema']}\nInvent and roleplay the required information for this process. Do NOT reply using json, instead reply with a single sentence instruction, that supplies most or all information that is mentioned in the schema."
        }],
        #tools=[function],
        #tool_choice="required",
        **default_or_options,
    )
    request = openrouter_response.choices[0].message.content.strip()
    #print(request)
    dialog.append({
        "role": "user",
        "content": request
    })

    corrections = 0
    while True:
        corrections += 1
        if corrections > 3:
            raise Exception('Too many corrections')
    
        openrouter_response: ChatCompletion = openrouter.chat.completions.create(
            model=model,
            messages=[{
                "role": "system",
                "content": f"Roleplay as an expert {entry['profession']} Assistant, the user is currently in the process of {entry['process']}. Your task is to call the specified tool on the users behalf. If you cannot perform the action due to lack of information, ask for clarification on required parameters in a single sentence question."
            }]+dialog,
            tools=[function],
            #tool_choice="required",
            **default_or_options,
        )

        if openrouter_response.choices[0].message.tool_calls:
            #print('tool_calls', openrouter_response.choices[0].message.tool_calls)
            dialog.append({"role":'assistant', "tool_calls": [
                {"id": call.id, "type": "function", "function": {"name": call.function.name, "arguments": call.function.arguments.strip()}} for call in openrouter_response.choices[0].message.tool_calls
            ]})
            break
        elif openrouter_response.choices[0].message.content:
            #print('Question', openrouter_response.choices[0].message.content)
            dialog.append({"role":'assistant', "content": openrouter_response.choices[0].message.content.strip()})
        else:
            print('Error', openrouter_response.choices[0].message)
            raise Exception('Error'+str(openrouter_response.choices[0].message))

        openrouter_response: ChatCompletion = openrouter.chat.completions.create(
            model=model,
            messages=[{
                "role": "system",
                "content": f"You are an expert {entry['profession']} currently in the process of {entry['process']}. Clarify any remaining questions by the user, by inventing detail. Your clarification should be short, in a single sentence, down to the point."
            }, {
                "role": "user",
                "content": f"How can I help you today?"
            }]+switch_roles(dialog)+[{
                "role": "user",
                "content": f"Please clarify what you mean."
            }],
            #tools=[function],
            #tool_choice="required",
            **default_or_options,
        )

        if not openrouter_response.choices[0].message.content:
            raise Exception('Error '+str(openrouter_response))
        #print('Clarification', openrouter_response.choices[0].message.content)
        dialog.append({"role":'user', "content": openrouter_response.choices[0].message.content.strip()})
    

    openrouter_response: ChatCompletion = openrouter.chat.completions.create(
        model=model,
        messages=[{
            "role": "system",
            "content": f"You are an expert {entry['profession']} currently in the process of {entry['process']}."
        }, {
            "role": "user",
            "content": f"I need you to do the following for me: {dialog[-1]['tool_calls'][0]}\nCome of with an improvised result that matches this input request. Call the {'generate_'+function_name+'_result'} function to generate a result."
        }],
        tools=[{
            "type": "function",
            "function": {
                "name": 'generate_'+function_name+'_result',
                "parameters": json.loads(entry['result_schema']),
            },
        }],
        tool_choice="required",
        **default_or_options,
    )

    #print('result', openrouter_response.choices[0].message)
    if openrouter_response.choices[0].message.tool_calls:
        dialog.append({
            "role":'tool', 
            "content": openrouter_response.choices[0].message.tool_calls[0].function.arguments.strip(), 
            "tool_call_id": dialog[-1]['tool_calls'][0]["id"]
        })
    else:
        dialog.append({
            "role":'tool', 
            "content": openrouter_response.choices[0].message.content.strip(), 
            "tool_call_id": dialog[-1]['tool_calls'][0]["id"]
        })

    openrouter_response: ChatCompletion = openrouter.chat.completions.create(
        model=model,
        messages=[{
            "role": "system",
            "content": f"You are an expert {entry['profession']} Assistant currently helping the user in the process of {entry['process']}. Confirm the successful execution to the user. You NEVER invent new information, only summarize the result, but omit any irrelevant or duplicate information. You NEVER mention what the user already knows from the previous dialog."
        }]+dialog,
        **default_or_options,
        temperature=1.2,
    )
    #print('reply', openrouter_response.choices[0].message.content)
    dialog.append({
        "role":'assistant', 
        "content": openrouter_response.choices[0].message.content.strip(), 
    })

    print(dialog)

    results.append({
        "profession": entry["profession"],
        "process": entry["process"],
        "name": entry["name"],
        "description": entry["description"],
        "parameter_schema": entry["parameter_schema"],
        "result_schema": entry["result_schema"],
        "dialog": dialog,
    })
    
    return results


processes = load_dataset("lucaelin/generic_domain_actions_v1", split="train[:1000]")
domain_actions = processes.map(
    sane_batch_map(generate_domain_action_dialog, {"profession": [], "process": [], "name": [], "description": [], "parameter_schema": [], "result_schema": [], "dialog": []}), 
    batched=True, 
    batch_size=1, 
    num_proc=6,
    remove_columns=processes.column_names
)
#domain_actions.sort(["profession", "process", "name"]).to_list()
domain_actions.sort(["profession", "process", "name"]).push_to_hub("lucaelin/generic_domain_action_dialogs_v1")

Map (num_proc=6):   0%|          | 0/1000 [00:00<?, ? examples/s]

[{'role': 'user', 'content': 'Carefully lower the vehicle to the ground by releasing the hydraulic pressure from the lift slowly, allowing the car to descend evenly onto its newly rotated tires, specifically the Goodyear Eagle F1s that were just swapped to their new positions on the 2018 Honda Civic Si.'}, {'role': 'assistant', 'tool_calls': [{'id': 'call_ZTQdpLxuWOs354Bihcs5dMHl', 'type': 'function', 'function': {'name': 'lower_vehicle', 'arguments': '{"type": "2018 Honda Civic Si"}'}}]}, {'role': 'tool', 'content': '{"lower_status": true}', 'tool_call_id': 'call_ZTQdpLxuWOs354Bihcs5dMHl'}, {'role': 'assistant', 'content': 'The vehicle has been successfully lowered to the ground, and the tire rotation for the 2018 Honda Civic Si is now complete, with the Goodyear Eagle F1 tires in their new positions.'}]


Map (num_proc=6):   0%|          | 1/1000 [00:04<1:20:45,  4.85s/ examples]

[{'role': 'user', 'content': '"Conduct individual counseling for client John Doe (client_id) on November 20th, 2022 (counseling_date), focusing on his recent life changes and stress management techniques (counseling_type: individual)."'}, {'role': 'assistant', 'tool_calls': [{'id': 'R8eBB5em1', 'type': 'function', 'function': {'name': 'providecounselingservices', 'arguments': '{"client_id": "John Doe", "counseling_type": "individual", "counseling_date": "2022-11-20"}'}}]}, {'role': 'tool', 'content': '{"action": "Provide Counseling Services", "client_id": "John Doe", "counseling_type": "individual"}', 'tool_call_id': 'R8eBB5em1'}, {'role': 'assistant', 'content': 'I have successfully helped facilitate the scheduling of an individual counseling session for client John Doe on November 20th, 2022. We will be focusing on discussing and addressing his recent life changes and providing guidance on effective stress management techniques.'}]


Map (num_proc=6):   0%|          | 2/1000 [00:06<52:35,  3.16s/ examples]  

...

[{'role': 'user', 'content': 'Create a new project titled "Digital Transformation Initiative" to oversee the company-wide migration to cloud-based software solutions, aimed at increasing operational efficiency and reducing costs.'}, {'role': 'assistant', 'tool_calls': [{'id': 'call-sZytBmpEtVqgcQCwY6zJXH', 'type': 'function', 'function': {'name': 'Create-Project', 'arguments': '{"name": "Digital Transformation Initiative"}'}}]}, {'role': 'tool', 'content': '{"project_id": "DTI-001"}', 'tool_call_id': 'call-sZytBmpEtVqgcQCwY6zJXH'}, {'role': 'assistant', 'content': 'Project "Digital Transformation Initiative" (DTI-001) created successfully.'}]


Map (num_proc=6): 100%|█████████▉| 999/1000 [57:17<02:03, 123.30s/ examples]

{'profession': 'Business Operations Manager', 'process': 'Resource Allocation', 'name': 'delete resource allocation', 'description': 'Delete the allocation of resources to a project.', 'parameter_schema': '{"type": "object", "properties": {"allocation_id": {"type": "string", "description": "The allocation id"}}}', 'result_schema': '{"type": "object", "properties": {"allocation_id": {"type": "string", "description": "The allocation id"}}}'}
Traceback (most recent call last):
  File "/home/luca/Dokumente/Elite-Dangerous-AI-Integration/train/utils.py", line 25, in batch_map
    results = results + fn(example)
                        ^^^^^^^^^^^
  File "/tmp/ipykernel_3741123/4215728129.py", line 192, in generate_domain_action_dialog
    "content": openrouter_response.choices[0].message.content.strip(),
              ~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
TypeError: 'NoneType' object is not subscriptable



Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.40s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/lucaelin/generic_domain_action_dialogs_v1/commit/99164ffe72ba6a2e015e72da4f91732076b5a18a', commit_message='Upload dataset', commit_description='', oid='99164ffe72ba6a2e015e72da4f91732076b5a18a', pr_url=None, pr_revision=None, pr_num=None)